In [ ]:
import numpy as np
from numpy import kron

# Single-qubit operators
I = np.array([[1,0],[0,1]], dtype=complex)
X = np.array([[0,1],[1,0]], dtype=complex)
Z = np.array([[1,0],[0,-1]], dtype=complex)
H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]], dtype=complex)

def op_on_qubit(op, which, n=3):
    """Apply single-qubit operator 'op' to the 1-based index 'which' in an n-qubit system."""
    M = None
    for i in range(1, n+1):
        M = kron(M, op if i==which else I) if M is not None else (op if i==which else I)
    return M

def CNOT(n, control, target):
    """n-qubit controlled-NOT (1-based indexing)."""
    P0 = np.array([[1,0],[0,0]], dtype=complex)
    P1 = np.array([[0,0],[0,1]], dtype=complex)
    # When control is |0⟩: identity. When control is |1⟩: apply X to target.
    return op_on_qubit(P0, control, n) + op_on_qubit(P1, control, n) @ op_on_qubit(X, target, n)

def format_state(state, tol=1e-9):
    labels = [format(i, '03b') for i in range(2**3)]
    out = []
    for amp, lab in zip(state, labels):
        if abs(amp) > tol:
            out.append(f"{amp:+.3f}|{lab}⟩")
    return "\n".join(out) if out else "0"


In [ ]:
# Bit-Flip Code
n = 3
psi = (1/np.sqrt(2))*np.array([1,1], dtype=complex)               # |+>
state = kron(kron(psi, np.array([1,0], dtype=complex)), np.array([1,0], dtype=complex))  # |ψ⟩|00⟩

# Encoding CNOT(1->2), CNOT(1->3)
state_enc = CNOT(n,1,2) @ (CNOT(n,1,3) @ state)

# Injecting X error on qubit 2
state_err = op_on_qubit(X,2,n) @ state_enc

# Stabilizers for syndrome
Z1Z2 = op_on_qubit(Z,1,n) @ op_on_qubit(Z,2,n)
Z1Z3 = op_on_qubit(Z,1,n) @ op_on_qubit(Z,3,n)
s1 = np.vdot(state_err, Z1Z2 @ state_err).real
s2 = np.vdot(state_err, Z1Z3 @ state_err).real

# Determining which qubit flipped based on s1,s2
if s1 < 0 and s2 > 0:
    fix = 2
elif s1 > 0 and s2 < 0:
    fix = 3
elif s1 < 0 and s2 < 0:
    fix = 1
else:
    fix = 0

# Applying correction
state_fix = op_on_qubit(X,fix,n) @ state_err if fix else state_err

# Decoding
state_out = CNOT(n,1,3) @ (CNOT(n,1,2) @ state_fix)

# Reduced density matrix of qubit 1 (should recover |+>)
rho = np.outer(state_out, np.conj(state_out))
rho1 = np.zeros((2,2), dtype=complex)
for a in range(2):
    for b in range(2):
        s = 0+0j
        for q2 in range(2):
            for q3 in range(2):
                i = (a<<2) | (q2<<1) | q3
                j = (b<<2) | (q2<<1) | q3
                s += rho[i,j]
        rho1[a,b] = s
rho_plus = np.outer(psi, np.conj(psi))
fid_plus = float(np.real(np.trace(rho1 @ rho_plus)))

print("[Encoded] (|000⟩+|111⟩)/√2\n", format_state(state_enc), "\n", sep="")
print("[After X error on qubit 2]\n", format_state(state_err), "\n", sep="")
print(f"Syndrome: <Z1Z2>={s1:.2f}, <Z1Z3>={s2:.2f}  →  fix qubit {fix if fix else 'none'}\n")
print("[After correction]\n", format_state(state_fix), "\n", sep="")
print("Reduced ρ₁:\n", np.round(rho1, 3))
print("Fidelity with |+⟩:", round(fid_plus, 6))

[Encoded] (|000⟩+|111⟩)/√2
+0.707+0.000j|000⟩
+0.707+0.000j|111⟩

[After X error on qubit 2]
+0.707+0.000j|010⟩
+0.707+0.000j|101⟩

Syndrome: <Z1Z2>=-1.00, <Z1Z3>=1.00  →  fix qubit 2

[After correction]
+0.707+0.000j|000⟩
+0.707+0.000j|111⟩

Reduced ρ₁:
 [[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
Fidelity with |+⟩: 1.0


In [ ]:
# Phase-Flip Code via H-rotation
HHH = op_on_qubit(H,1,3) @ op_on_qubit(H,2,3) @ op_on_qubit(H,3,3)

# Reusing |ψ>|00⟩ and the encoder
state = kron(kron(psi, np.array([1,0], dtype=complex)), np.array([1,0], dtype=complex))
state_enc = CNOT(n,1,2) @ (CNOT(n,1,3) @ state)

# Injecting Z error on qubit 2
state_err = op_on_qubit(Z,2,3) @ state_enc

# Rotating so Z-error looks like X-error
rot = HHH @ state_err

Z1Z2 = op_on_qubit(Z,1,3) @ op_on_qubit(Z,2,3)
Z1Z3 = op_on_qubit(Z,1,3) @ op_on_qubit(Z,3,3)
s1 = np.vdot(rot, Z1Z2 @ rot).real
s2 = np.vdot(rot, Z1Z3 @ rot).real

if s1 < 0 and s2 > 0:
    fix = 2
elif s1 > 0 and s2 < 0:
    fix = 3
elif s1 < 0 and s2 < 0:
    fix = 1
else:
    fix = 0

# Correcting in rotated frame, then rotating back
rot_fix = op_on_qubit(X,fix,3) @ rot if fix else rot
state_fix = HHH @ rot_fix

# Decoding
state_out = CNOT(n,1,3) @ (CNOT(n,1,2) @ state_fix)

# Reduced density matrix of qubit 1 and fidelity with |+>
rho = np.outer(state_out, np.conj(state_out))
rho1 = np.zeros((2,2), dtype=complex)
for a in range(2):
    for b in range(2):
        s = 0+0j
        for q2 in range(2):
            for q3 in range(2):
                i = (a<<2) | (q2<<1) | q3
                j = (b<<2) | (q2<<1) | q3
                s += rho[i,j]
        rho1[a,b] = s
fid_plus = float(np.real(np.trace(rho1 @ rho_plus)))

print("Rotate with H⊗H⊗H, measure syndrome, fix, rotate back.\n")
print(f"Syndrome: <Z1Z2>={s1:.2f}, <Z1Z3>={s2:.2f}  →  fix qubit {fix if fix else 'none'}\n")
print("Reduced ρ₁:\n", np.round(rho1, 3))
print("Fidelity with |+⟩:", round(fid_plus, 6))

Rotate with H⊗H⊗H, measure syndrome, fix, rotate back.

Syndrome: <Z1Z2>=0.00, <Z1Z3>=0.00  →  fix qubit none

Reduced ρ₁:
 [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]
Fidelity with |+⟩: 0.0
